In [1]:
using OceananigansLagrangianFilter
using Printf
using JLD2
using Oceananigans.Units: Time
using CairoMakie
using CUDA
using Oceananigans.TimeSteppers: reset!

Precompiling packages...
   4497.7 ms  ✓ OceananigansLagrangianFilter
  1 dependency successfully precompiled in 6 seconds. 357 already precompiled.


In [2]:
# User defined options

original_data_filename = joinpath(@__DIR__, "SW_vort_with_tracer.jld2")
T_start = 0
T_end = 40

arch = GPU()

# Set the output period
T_out = 0.1

# Set filter order and cut-off frequency
# Amplitude of frequency response of filter will be squared Butterworth order 2^N
N = 2 
freq_c = 1

# Define variables to filter
original_var_names = ("ω","T")

# Define velocities to use for filtering
velocity_names = ("u","v")

# Set filtering parameters (this is for Butterworth-type, could define others here)
filter_params = set_BW_filter_params(N=N,freq_c=freq_c)

# Set the time step for the simulation
Δt = 1e-3

# Decide whether to solve for and output maps to generalised Lagrangian mean
map_to_mean = true

# Name output files
forward_output_filename = joinpath(@__DIR__, "forward_LF_freqc_1_with_tracer.jld2")
backward_output_filename = joinpath(@__DIR__, "backward_LF_freqc_1_with_tracer.jld2")
combined_output_filename = joinpath(@__DIR__, "combined_LF_freqc_1_with_tracer.jld2")
# Manipulate data on disk to have correct order
T = set_data_on_disk!(original_data_filename, direction="forward", T_start = T_start, T_end = T_end)

Current direction is forward
No need to reverse order of data


40

In [ ]:
# Load in saved data from simulation
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))
println("Loaded data from $original_data_filename")

# Create the original variables - these will be auxiliary fields in the model
original_vars = create_original_vars(original_var_names, grid)
println("Created original variables: ", original_vars)

# Create the filtered variables - these will be tracers in the model
filtered_vars = create_filtered_vars(original_var_names, velocity_names, filter_params, map_to_mean=map_to_mean)
println("Created filtered variables: ", filtered_vars)

# Create forcing for these filtered variables
forcing = create_forcing(filtered_vars, original_var_names, velocity_names, filter_params)
println("Created forcing for filtered variables ")

# Define model 
model = LagrangianFilter(;grid, tracers = filtered_vars, auxiliary_fields = original_vars, forcing = forcing, advection=WENO())
println("Created model")

# Define our outputs # 
filtered_outputs = create_output_fields(model, original_var_names, velocity_names, filter_params)
println("Defined outputs")

# Define the filtering simulation 
simulation = Simulation(model, Δt = Δt, stop_time = T) 
println("Defined simulation")

# Tell the simulation to use the saved data
simulation.callbacks[:update_input_data] = Callback(update_input_data!, parameters = (velocities = saved_velocities, original_vars = saved_original_vars))

# Add a progress monitor
function progress(sim)
    @info @sprintf("Simulation time: %s, max(|u|):%.2e \n", 
                   prettytime(sim.model.clock.time), 
                   maximum(abs, model.velocities.u))             
     return nothing
 end

simulation.callbacks[:progress] = Callback(progress, IterationInterval(50))

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = forward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

run!(simulation)

Loaded data from /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/turbulent_SW/SW_vort_with_tracer.jld2
Created original variables: NamedTuple with 2 Fields on 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo:
├── T: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
└── ω: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
Created filtered variables: (:ωC1, :ωC2, :TC1, :TC2, :xi_u_C1, :xi_u_C2, :xi_v_C1, :xi_v_C2, :ωS1, :ωS2, :TS1, :TS2, :xi_u_S1, :xi_u_S2, :xi_v_S1, :xi_v_S2)
Created forcing for filtered variables 
Created model
Defined outputs
Defined simulation


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):9.69e-01 
[ Info:     ... simulation initialization complete (2.592 minutes)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (16.897 seconds).
[ Info: Simulation time: 50.000 ms, max(|u|):9.55e-01 
[ Info: Simulation time: 100.000 ms, max(|u|):9.92e-01 
[ Info: Simulation time: 148.000 ms, max(|u|):9.70e-01 
[ Info: Simulation time: 198.000 ms, max(|u|):1.01e+00 
[ Info: Simulation time: 248.000 ms, max(|u|):9.73e-01 
[ Info: Simulation time: 298.000 ms, max(|u|):1.01e+00 
[ Info: Simulation time: 348.000 ms, max(|u|):9.83e-01 
[ Info: Simulation time: 398.000 ms, max(|u|):1.01e+00 
[ Info: Simulation time: 448.000 ms, max(|u|):1.02e+00 
[ Info: Simulation time: 498.000 ms, max(|u|):1.07e+00 
[ Info: Simulation time: 548 ms, max(|u|):1.07e+00 
[ Info: Simulation time: 598.000 ms, max(|u|):1.11e+00 
[ Info: Simulation time: 648.000 ms, max(|u|):1.11e+00 
[ Info: Simulation t

In [15]:

# Now, run it backwards. Switch the data direction on disk
T = set_data_on_disk!(original_data_filename, direction="backward", T_start = T_start, T_end = T_end)

# Reload the saved data 
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))

# Reinitialize tracers to zero (for consistency)
kwargs = (; (tracer => 0 for tracer in keys(model.tracers))...)
set!(model; kwargs...)  

reset!(model.clock)

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = backward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

# And run the backward simulation.
run!(simulation)


Current direction is forward
Reversing order of data
Reversed order of ω
Reversed order of and switched sign of u
Reversed order of and switched sign of v
Reversed order of T
Reversed order of t_simulation
Reversed order of and shifted t
New direction is backward


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):9.70e-01 
[ Info:     ... simulation initialization complete (90.564 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (13.180 ms).
[ Info: Simulation time: 50.000 ms, max(|u|):8.53e-01 
[ Info: Simulation time: 100.000 ms, max(|u|):9.23e-01 
[ Info: Simulation time: 148.000 ms, max(|u|):8.58e-01 
[ Info: Simulation time: 198.000 ms, max(|u|):9.45e-01 
[ Info: Simulation time: 248.000 ms, max(|u|):9.81e-01 
[ Info: Simulation time: 298.000 ms, max(|u|):1.14e+00 
[ Info: Simulation time: 348.000 ms, max(|u|):1.17e+00 
[ Info: Simulation time: 398.000 ms, max(|u|):1.31e+00 
[ Info: Simulation time: 448.000 ms, max(|u|):1.30e+00 
[ Info: Simulation time: 498.000 ms, max(|u|):1.33e+00 
[ Info: Simulation time: 548 ms, max(|u|):1.26e+00 
[ Info: Simulation time: 598.000 ms, max(|u|):1.26e+00 
[ Info: Simulation time: 648.000 ms, max(|u|):1.14e+00 
[ Info: Simulation time: 698.

In [20]:
sum_forward_backward_contributions!(combined_output_filename, forward_output_filename, backward_output_filename, T, velocity_names, original_var_names)

Combined forward and backward contributions into /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/turbulent_SW/combined_LF_freqc_1_with_tracer.jld2


In [3]:
npad = 5
regrid_to_mean_position!(combined_output_filename, original_var_names, velocity_names, npad)


0.0%┣                                              ┫ 0/401 [00:00<00:-1, -0s/it]
0.2%┣                                          ┫ 1/401 [00:05<Inf:Inf, InfGs/it]
0.5%┣▎                                              ┫ 2/401 [00:06<38:07, 6s/it]
0.7%┣▍                                              ┫ 3/401 [00:07<22:27, 3s/it]
1.0%┣▌                                              ┫ 4/401 [00:08<16:46, 3s/it]
1.2%┣▋                                              ┫ 5/401 [00:08<13:51, 2s/it]
1.5%┣▊                                              ┫ 6/401 [00:09<12:05, 2s/it]
1.7%┣▉                                              ┫ 7/401 [00:10<10:54, 2s/it]
2.0%┣█                                              ┫ 8/401 [00:11<10:03, 2s/it]
2.2%┣█                                              ┫ 9/401 [00:12<09:25, 1s/it]
2.5%┣█▏                                            ┫ 10/401 [00:12<09:00, 1s/it]
2.7%┣█▎                                            ┫ 11/401 [00:13<08:39, 1s/it]
3.0%┣█▍                     

In [36]:
# Animate
timeseries1 = FieldTimeSeries(combined_output_filename, "T")
timeseries2 = FieldTimeSeries(combined_output_filename, "T_filtered")
timeseries3 = FieldTimeSeries(combined_output_filename, "T_filtered_regrid")
times = timeseries1.times

set_theme!(Theme(fontsize = 20))
fig = Figure(size = (1000, 500))

axis_kwargs = (xlabel = "x",
               ylabel = "y",
               limits = ((0, 2π), (0, 2π)),
               aspect = AxisAspect(1))

ax1 = Axis(fig[2, 1]; title = "Raw vorticity", axis_kwargs...)
ax2 = Axis(fig[2, 2]; title = "Filtered vorticity", axis_kwargs...)
ax3 = Axis(fig[2, 3]; title = "Filtered regridded vorticity", axis_kwargs...)


n = Observable(1)
Observable(1)

var1 = @lift timeseries1[$n]
var2 = @lift timeseries2[$n]
var3 = @lift timeseries3[$n]

heatmap!(ax1, var1; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax2, var2; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax3, var3; colormap = :balance, colorrange = (-1, 1))


title = @lift "t = " * string(round(times[$n], digits=2))
Label(fig[1, 1:3], title, fontsize=24, tellwidth=false)

fig

frames = 1:length(times)

@info "Making an animation"

CairoMakie.record(fig, "LF_movie_freq_c_1_T_pycall.mp4", frames, framerate=24) do i
    n[] = i
end

[ Info: Making an animation


"LF_movie_freq_c_1_T_pycall.mp4"

In [4]:
jld2_to_netcdf(combined_output_filename, joinpath(@__DIR__, "combined_output_freq_c_1_tracer_WENO_pycall_5_pad.nc"))

closed Dataset

In [15]:
close(file)

In [4]:
file = jldopen(combined_output_filename, "r")
grid = file["serialized/grid"]
propertynames(grid)

(:architecture, :Nx, :Ny, :Nz, :Hx, :Hy, :Hz, :Lx, :Ly, :Lz, :Δxᶠᵃᵃ, :Δxᶜᵃᵃ, :xᶠᵃᵃ, :xᶜᵃᵃ, :Δyᵃᶠᵃ, :Δyᵃᶜᵃ, :yᵃᶠᵃ, :yᵃᶜᵃ, :z)

In [20]:
grid.xᶠᵃᵃ[1]

-1.9141776286644982e-18

In [ ]:
grid.xᶠᵃᵃ[grid.Nx+1]

1.0